In [ ]:
!pip install contractions
!pip install textsearch
!pip install tqdm


In [ ]:
import nltk
nltk.download('punkt')


from tensorflow import keras
import pandas as pd
import numpy as np

import time
import datetime


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import contractions
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:

from tensorflow.keras.layers import GRU, Embedding, Flatten, Input, Dense, LeakyReLU, BatchNormalization, Reshape
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

seed = 3541
np.random.seed(seed)


def date_time(x):
    if x == 1:
        return 'Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())
    if x == 2:
        return 'Timestamp: {:%Y-%b-%d %H:%M:%S}'.format(datetime.datetime.now())
    if x == 3:
        return 'Date now: %s' % datetime.datetime.now()
    if x == 4:
        return 'Date today: %s' % datetime.date.today()

In [ ]:

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def pre_process_corpus(docs):
    norm_docs = []
    for doc in tqdm.tqdm(docs):
        doc = strip_html_tags(doc)
        doc = doc.translate(doc.maketrans("\n\t\r", "   "))
        doc = doc.lower()
        doc = remove_accented_chars(doc)
        doc = contractions.fix(doc)

        doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
        doc = re.sub(' +', ' ', doc)
        doc = doc.strip()
        norm_docs.append(doc)

    return norm_docs

In [ ]:

modelAmazon = keras.models.load_model('/content/Binary_Classification_86_Amazon_Reviews_CNN.h5')


modelYelp = keras.models.load_model('/content/Binary_Classification_90_Yelp_Reviews_CNN.h5')


dataset_test_yelp = pd.read_csv('/content/yelp_test.csv')


dataset_test_Amazon = pd.read_csv('/content/amazon_test.csv')

dataset_train_Amazon = pd.read_csv('/content/amazon_review_sa_binary_csv/train.csv')


In [ ]:

train_Amazon = dataset_train_Amazon.iloc[:100,:]

test_Amazon = test_Amazon.iloc[:38000,:]


In [ ]:
# Taking only necessary columns
X_test_yelp = test_yelp['review_text'].values
y_test_yelp = test_yelp['class_index'].values
X_test_Amazon = test_Amazon['review_text'].values
y_test_Amazon = test_Amazon['class_index'].values
X_train_Amazon = train_Amazon['review_text'].values
y_train_Amazon = train_Amazon['class_index'].values
X_M_data = M_data['review_text'].values
y_M_data = M_data['class_index'].values


In [ ]:
# Preprocessing the Data
X_test_yelp = pre_process_corpus(X_test_yelp)
X_test_Amazon = pre_process_corpus(X_test_Amazon)
X_train_Amazon = pre_process_corpus(X_train_Amazon)
X_M_data = pre_process_corpus(X_M_data)


In [ ]:
# Creating and Fitting the Tokenizer
t = Tokenizer(oov_token='<UNK>')
t.fit_on_texts(X_train_Amazon)
t.word_index['<PAD>'] = 0

In [ ]:
# Transforming text to sequences
X_test_yelp = t.texts_to_sequences(X_test_yelp)
X_test_Amazon = t.texts_to_sequences(X_test_Amazon)
X_train_Amazon = t.texts_to_sequences(X_train_Amazon)
X_M_data = t.texts_to_sequences(X_M_data)


In [ ]:
# Padding the transformed text (sentences) to maximum length of 220
X_test_yelp = sequence.pad_sequences(X_test_yelp, maxlen=220)
X_test_Amazon = sequence.pad_sequences(X_test_Amazon, maxlen=220)
X_train_Amazon = sequence.pad_sequences(X_train_Amazon, maxlen=220)
X_M_data = sequence.pad_sequences(X_M_data, maxlen=220)


In [ ]:
# Creating and Fitting the label encoder
le = LabelEncoder()
num_classes=2
y_train_Amazon = le.fit_transform(y_train_Amazon)


In [ ]:
# Transforming the labels
y_test_yelp = le.transform(y_test_yelp)
y_test_Amazon = le.transform(y_test_Amazon)
y_M_data = le.transform(y_M_data)

In [ ]:
# Evaluating the models
# Testing Amazon Classifier on Yelp Test Data
print("Testing Amazon Classifier on Yelp Test Data")
scores = modelAmazon.evaluate(X_test_yelp, y_test_yelp, verbose=1)
print("Accuracy: %.2f%% /n" % (scores[1]*100))


In [ ]:
# Testing Amazon Classifier on Amazon Test Data
print(" Testing Yelp Classifier on Amazon Test Data")
scores = modelAmazon.evaluate(X_test_Amazon, y_test_Amazon, verbose=1)
print("Accuracy: %.2f%% /n" % (scores[1]*100))

In [ ]:
# Testing Yelp Classifier on Yelp Test Data
print(" Testing Yelp Classifier on Yelp Test Data")
scores = modelYelp.evaluate(X_test_yelp, y_test_yelp, verbose=1)
print("Accuracy: %.2f%%  " % (scores[1]*100))

In [ ]:
# Testing Yelp Classifier on Amazon Test Data
print(" Testing Yelp Classifier on Amazon Test Data")
scores = modelYelp.evaluate(X_test_Amazon, y_test_Amazon, verbose=1)
print("Accuracy: %.2f%% " % (scores[1]*100))

In [ ]:
# Evaluating the model on manual Data
# Testing Yelp Classifier on Manually Inputted Reviews
print(" Testing Yelp Classifier on Manually Inputted Reviews ")
scores = modelYelp.evaluate(X_M_data, y_M_data, verbose=1)
print("Accuracy: %.2f%% " % (scores[1]*100))

In [ ]:

# Testing Yelp Classifier on Manually Inputted Reviews
print(" Testing Yelp Classifier on Manually Inputted Reviews")
scores = modelAmazon.evaluate(X_M_data, y_M_data, verbose=1)
print("Accuracy: %.2f%% " % (scores[1]*100))